<a href="https://colab.research.google.com/github/Pengwei-Yang/NLP/blob/main/Dependency_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependency Parsing Exercise

Let's parse the following sentence!

*  I prefer the morning flight through Denver

Do this parsing exercise (probably easiest to do on paper) and ***compare with the answer (at the end of this lab)***. Note - you do not need to include labels in your parse. We have provided them in the answer just for your information.

In places where your answer is different, think about whether it is a mistake, or whether there are two ways of interpreting the sentence and you chose a different one than the one we provide.

For information about how to do dependency parsing, see the lecture 7 recording.


## Off-the-Shelf Tool (spaCy Parser)
Now let's use [spaCy](https://spacy.io/) to automatically parse sentences.
SpaCy provides several parsing models, some based on **neural networks and some based on linear models**.

Parse the sentence ***'The woman showed a wellshaped smile in the dark.'*** (note, the typo, combining 'well' and 'shaped' is intentional, to match data we are using later)


In [ ]:
import spacy
#PrettyTable is a Python library for generating attractive ASCII tables.
from prettytable import PrettyTable

#load the spacy api with the pre-trained statistical models for English. English multi-task CNN trained on the OntoNotes corpus
nlp = spacy.load("en_core_web_sm")

#define the sentence
sentence = "The woman showed a wellshaped smile in the dark."
parse = nlp(sentence) # This step is using machine learning algorithm to get the dependency types, and the arc

x = PrettyTable()
#define column names
x.field_names = ["TokenID", "Token", "HeadID", "Dependency"]

#spaCy does not provide the fake ROOT token so add a row for the fake Root
x.add_row([0,"ROOT",0,"-"])

#recording the dependency for each token
for token in parse:
  if token.dep_=="ROOT":
    x.add_row([token.i+1,token.text,"0",token.dep_])
  else:
    x.add_row([token.i+1,token.text,token.head.i+1,token.dep_])

print("Parsing Result with Spacy API")
#printing the table
print(x)

Parsing Result with Spacy API
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    ROOT    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    2    |   woman    |   3    |   nsubj    |
|    3    |   showed   |   0    |    ROOT    |
|    4    |     a      |   6    |    det     |
|    5    | wellshaped |   6    |    amod    |
|    6    |   smile    |   3    |    dobj    |
|    7    |     in     |   3    |    prep    |
|    8    |    the     |   9    |    det     |
|    9    |    dark    |   7    |    pobj    |
|    10   |     .      |   3    |   punct    |
+---------+------------+--------+------------+


In [ ]:
for token in parse:
  print(token.dep_) # Gives the dependency

det
nsubj
ROOT
det
amod
dobj
prep
det
pobj
punct


## Parsing Visualisation
Spacy includes a built-in visualisation suite, [displaCy](https://spacy.io/api/top-level#displacy). It makes it easier to see the dependency structure. In this case, the output is mostly correct, except for 'in' and 'dark'. The head of 'in' should be 'dark', and the head of 'dark' should be 'showed'.

In [ ]:
#spaCy comes with a built-in visualisation suite
from spacy import displacy

doc = nlp(sentence)
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})


### How can we evaluate the parsing result?

We compare the output with a gold standard set of annotations produced by people.

[Universal Dependencies](http://universaldependencies.org/) is a large project with many researchers working to create a consistent set of dependency annotations for many different languages and types of text.

## Parsing Evaluation (with Universal Dependencies)

The following code shows a function for evaluating the performance of the spaCy parser on Universal Dependencies data.

First we need to install a library to process that data format used by Universal Dependencies: [CoNLL-U](http://universaldependencies.org/format.html).



In [ ]:
#CoNLL-U Parser parses a CoNLL-U formatted string into a nested python dictionary.
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Next, we'll define an evaluation function to compare predicted and gold dependency parses.

In [ ]:
import numpy as np
import conllu as conllu
from conllu import parse

#text: the text that you would like to parse
#gold: the gold-standard annotation for the text
def evaluate_sentence_parsing(text, gold):
  #parse the text with spaCy API
  parse = nlp(text)
  #setup the table for the parsing result table
  x = PrettyTable()
  x.field_names = ["TokenID", "Token", "HeadID", "Dependency"]
  x.add_row([0,"root",0,"-"])

  for token in parse:
    if token.dep_=="ROOT":
      x.add_row([token.i+1,token.text,"0","root"])
      token.dep_="root"
    else:
      x.add_row([token.i+1,token.text,token.head.i+1,token.dep_])

  #printing the parsing result
  print("Parsing Result with Spacy API")
  print(x)

  #setup the table for the parsing result table
  y = PrettyTable()
  y.field_names = ["TokenID", "Token", "HeadID", "Dependency"]
  y.add_row([0,"root",0,"-"])

  #read the gold-standard with the conll-u library. Note, it says 'parse' here,
  #but we are not running a syntactic parser, this is 'parse' in the sense of 'process an input file'.
  sentences = conllu.parse(gold)
  sentence = sentences[0]

  for token in sentence:
    if token['deprel']=="root":
      y.add_row([token['id'],token['form'],"0",token['deprel']])
    else:
      y.add_row([token['id'],token['form'],token['head'],token['deprel']])
  print("Gold-Standard Annotation")
  print(y)

  #summarise the list of predicted heads
  pred_head = [t.head.i+1 if i != t.head.i else 0 for i, t in enumerate(parse)]
  #summarise the list of gold heads
  gold_head = [token['head'] for i, token in enumerate(sentence)]

  #summarise the list of predicted dependency relations
  pred_dep = [t.dep_ for i, t in enumerate(parse)]
  #summarise the list of gold dependency relations
  gold_dep = [token['deprel'] for i, token in enumerate(sentence)]

  print("\n\nParsing Result vs Gold - Head")
  print(pred_head)
  print(gold_head)

  print("\n\nParsing Result vs Gold - Dependency")
  print(pred_dep)
  print(gold_dep)

  #performance evaluation - Unlabeled Attachment Score (UAS)
  #Unlabeled Attachment Score (UAS): the percent of words that have the correct heads
  uas_accuracy = np.sum([1 if g == p else 0 for g, p in zip(gold_head, pred_head)]) / len(gold_head)


  print("\n\nUnlabeled Attachment Score (UAS)")
  return uas_accuracy

With the above function, let's try to test and evaluate it.
There are two evaluation metrics:
1. **Unlabeled Attachment Score (UAS)**: the percent of words that have the correct heads
2. **Labeled Attachment Score (LAS)**: the percent of words that have the correct heads and labels



In [ ]:
#testing sentence
text = "The woman showed a wellshaped smile in the dark."

#gold-standard of the sentence
gold = """
# text = The woman showed a wellshaped smile in the dark.
1	The	the	DET	DEF	Definite=Def|PronType=Art	2	det	_	_
2	woman	woman	NOUN	SG-NOM	Number=Sing	3	nsubj	_	_
3	showed	show	VERB	PAST	Mood=Ind|Tense=Past|VerbForm=Fin	0	root	_	_
4	a	a	DET	IND-SG	Definite=Ind|PronType=Art	6	det	_	_
5	wellshaped	wellshaped	ADJ	POS	Degree=Pos	6	amod	_	_
6	smile	smile	NOUN	SG-NOM	Case=Nom	3	obj	_	_
7	in	in	ADP	_	_	9	prep	_	_
8	the	the	DET	DEF	Definite=Def|PronType=Art	9	det	_	_
9	dark	dark	ADJ	POS	Degree=Pos	3	obl	_	SpaceAfter=No
10	.	.	PUNCT	Period	_	3	punct	_	_


"""

print(evaluate_sentence_parsing(text, gold))

Parsing Result with Spacy API
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    root    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    2    |   woman    |   3    |   nsubj    |
|    3    |   showed   |   0    |    root    |
|    4    |     a      |   6    |    det     |
|    5    | wellshaped |   6    |    amod    |
|    6    |   smile    |   3    |    dobj    |
|    7    |     in     |   3    |    prep    |
|    8    |    the     |   9    |    det     |
|    9    |    dark    |   7    |    pobj    |
|    10   |     .      |   3    |   punct    |
+---------+------------+--------+------------+
Gold-Standard Annotation
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    root    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    

###Summary
The result shows that the UAS is 0.8.

It means that there are 2 incorrectly predicted heads out of 10.
1.   The gold head of token 'in' is 'dark', whereas the prediction is 'smile'
2.   The gold head of token 'dark' is 'showed', whereas the prediction is 'in'.

**UAS** is the percent of words that have the correct heads

**LAS** is the percent of words that have the correct heads and labels.


---



## Transition-based Dependency Parser

A Transition-based Dependency Parser is a simple form of dependency parser. It builds a tree in a left-to-right sweep over the input.


The transition-based model consists of:
*   a ***buffer***, for storing unprocessed text. At the beginning, it stores the input tokens.
*   a ***stack***, for storing elements being processed. At the beginning, it has a ROOT token.
*   a list of ***dependency relations***, you can also consider this as the parsing result. It is usually a list of tuples, each of which is a token and its head.

The ***arc-standard*** algorithm has three actions which can be applied to change the states of buffer, stack and dependency relation list:

*   **LEFTARC**: create a head-dependent relation between the word at the top of the stack and the word directly beneath it; remove the lower word from the stack.
*   **RIGHTARC**: create a head-dependent relation between the second word on the stack and the word at the top; remove the word at the top of the stack;
*   **SHIFT**: remove the word from the front of the input buffer and push it onto the stack.

####Lecture 7 - slide 60
![initial dependency parsing state](https://drive.google.com/uc?export=view&id=1nctCvQ8rlBaM-NdVywh0i0HBYRspqtn2)





In [ ]:
class Parse(object):
    def __init__(self, sentence): # All of the items are stored as lists
        self.sentence = sentence
        self.stack = ["<ROOT>"]
        self.buffer = sentence.split()
        self.relations = []

   #transition setup
    def parse_step(self, action):
        assert action in ["SHIFT", "LEFTARC", "RIGHTARC"]
        if action == "SHIFT":
            assert len(self.buffer) > 0
            token = self.buffer.pop(0)
            self.stack.append(token)
        else:
            assert len(self.stack) >= 2
            if action == "LEFTARC":
                relation = (self.stack[-1], self.stack[-2]) # Pop the child of the relation
                self.relations.append(relation)
                self.stack.pop(-2)
            else:
                relation = (self.stack[-2], self.stack[-1])
                self.relations.append(relation)
                self.stack.pop(-1)

    def parse(self, actions):
        print("Let's start:")
        output_parse_state(self)
        print("*" * 50)
        for action in actions:
            self.parse_step(action)
            print("after action:", action)
            output_parse_state(self)
            print("*" * 50)


def output_parse_state(parse):
    print("Stack:", " ".join(parse.stack))
    print("Buffer:", " ".join(parse.buffer))
    print("Relations:")
    for relation in parse.relations:
        print("  %s -> %s" % (relation[0], relation[1]))

In [ ]:
sentence = "Book me the morning flight"

actions = ["SHIFT", "SHIFT", "RIGHTARC", "SHIFT", "SHIFT", "SHIFT", "LEFTARC",
           "LEFTARC", "RIGHTARC", "RIGHTARC"]

parse_obj = Parse(sentence)
parse_obj.parse(actions)

Let's start:
Stack: <ROOT>
Buffer: Book me the morning flight
Relations:
**************************************************
after action: SHIFT
Stack: <ROOT> Book
Buffer: me the morning flight
Relations:
**************************************************
after action: SHIFT
Stack: <ROOT> Book me
Buffer: the morning flight
Relations:
**************************************************
after action: RIGHTARC
Stack: <ROOT> Book
Buffer: the morning flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the
Buffer: morning flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the morning
Buffer: flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the morning flight
Buffer: 
Relations:
  Book -> me
**************************************************
after action: LEFTARC
Stack: <ROOT> Book the f

###Discussion
Where does this list of actions come from? It looks magic, but how can I know what actions to apply? A classifier is trained to predict what the next action is given the current state.
This machine learning classifier takes the state of buffer, stack, previous list of relations as input, and predict what the next action is.

For more details, please read ***Lecture 7 from slide 76***

# Dependency Parsing Answer

This the answer for the question at the start of the lab. We have included labels, but you do not need to.

![alt text](https://www.researchgate.net/publication/334216098/figure/fig4/AS:776769690951681@1562207734172/A-sentence-in-dependency-format-Source-19-p-245.jpg)

# Exercise

## E1. Multiple Choice Questions

Please answer the two questions below. Note - you must get both right to get a point.

### Question 1
The following diagram shows the transition probabilities for a Markov model for Part-of-Speech (POS) tags:

![initial dependency parsing state](https://drive.google.com/uc?export=view&id=1bHV4i7OZdhMpylEaUpxZGvHIkavDF6BC)

The image can be found at: <https://drive.google.com/file/d/1bHV4i7OZdhMpylEaUpxZGvHIkavDF6BC/view?usp=share_link>. The values it shows can also be read in this table:

From, To ->    | DT | RB | JJ | NN
------|----|----|----|----
START | 0.6 | 0.15 | 0 | 0.25
DT    | 0 | 0.1 | 0.3 | 0.6
RB    | 0 | 0 | 0.6 | 0.4
JJ    | 0 | 0.05 | 0 | 0.95
NN    | 0.05 | 0.75 | 0.2 | 0

For example, the edge from DT to RB has a value of 0.1.

Suppose that you need to evaluate the phrase ‘The NLP’. Unfortunately, you do not know the POS tag for "NLP". **If you do know that "the" had the tag "DT"**, then, according to the transition probabilities above, which of the following are possible tag sequences for the phrase "The NLP"? (provide all that are possible)

1. DT, RB, JJ
2. DT, JJ
3. DT, RB
4. RB, NN
5. DT, NN
6. DT, NN, RB
7. DT, JJ, JJ
8. NN, JJ

Answer:
**2, 3, 5**


### Question 2

Two popular dependency parsing approaches are: 1) graph-based and 2) transition-based. Which of the statements below are CORRECT?

1. The transition-based parser makes a series of sequential decisions, whereas the graph-based parser uses an algorithm to decide the entire parse simultaneously.
2. Transition-based parsers can use neural models to score edges, but graph-based parsers cannot.
3. Transition-based and graph-based parsers can both produce any projective dependency parse.
4. UAS is always greater than or equal to LAS.

Answer: **1, 3, 4 are correct.**

**2 is incorrect since both transition-based and graph-based parsers can use neural models to score edges or make parsing decisions.**

##E2. SpaCy Parser Practice

You are going to write a function which takes the index of a sentence as input and return the **LAS** of the output produced by the spaCy model, you should also **visualise the parsing graph** produced by the spaCy model.

The data we will use is from Universal Dependencies. For details about LAS, see Lecture 7, slides 78-80.

First, we provide some code to download and prepare the data:

In [ ]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-Atis/master/en_atis-ud-dev.conllu

!pip install conllu
from io import open
from conllu import parse_incr
import spacy
import numpy as np
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
data_file = open("en_atis-ud-dev.conllu", "r", encoding="utf-8")

sent_list = []
gold_head_list = []
gold_label_list = []
n = 0

for tokenlist in parse_incr(data_file):
    n += 1
    if n > 20:
        break

    sent = tokenlist.metadata['text']
    gold_head = [token['head'] for i, token in enumerate(tokenlist)]
    gold_label = [token['deprel'] for i, token in enumerate(tokenlist)]
    sent_list.append(sent)
    gold_head_list.append(gold_head)
    gold_label_list.append(gold_label)

print(sent_list[1])
print(gold_head_list[1])
print(gold_label_list[1])

--2023-04-21 02:26:27--  https://raw.githubusercontent.com/UniversalDependencies/UD_English-Atis/master/en_atis-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349562 (341K) [text/plain]
Saving to: ‘en_atis-ud-dev.conllu’

en_atis-ud-dev.conl 100%[===================>] 341.37K  --.-KB/s    in 0.007s  

2023-04-21 02:26:27 (47.6 MB/s) - ‘en_atis-ud-dev.conllu’ saved [349562/349562]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
i want a flight from memphis to seattle that arrives no later than 3 pm
[2, 0, 4, 2, 6, 4, 8, 4, 10, 4, 12, 10, 15, 15, 12]
['nsubj', 'root', 'det', 'obj', 'case', 'nmod', 'case', 'nmod', 'nsubj', 'acl:relcl', 'det', 'obl', 'case', 'nummod', 'nmod']


Now, please write y

**Note:** Please DO NOT change the code outside of the parse_LAS function

In [ ]:
def parse_LAS(index_sent):
    sent = sent_list[index_sent]
    gold_head = gold_head_list[index_sent]
    gold_label = gold_label_list[index_sent]
    # Please complete this part
    parsed = nlp(sent)
    pred_head = [t.head.i+1 if i != t.head.i else 0 for i, t in enumerate(parsed)]
    pred_label = [token.dep_ for token in parsed]
    las_accuracy = np.sum([1 if g == p and gl == pl else 0 for g, p, gl, pl in zip(gold_head, pred_head, gold_label, pred_label)]) / len(gold_head)

    return parsed, las_accuracy


parsed, LAS = parse_LAS(1)
print("LAS: {0:.4f}".format(LAS))
displacy.render(parsed, style='dep', jupyter=True, options={'distance':90})

LAS: 0.2667
